In [1]:
import numpy as np
import pandas as pd
import sqlite3
import json
import glob
import gensim
import tqdm as tq
from tqdm import tqdm
import scipy
import gc
import pickle
import os
import collections

import string
import re
import MeCab
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors

import torch
from transformers import BertJapaneseTokenizer, BertModel, BertForMaskedLM
from transformers import BertConfig
from transformers import BertForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import TrainingArguments
from transformers import Trainer
import datasets

# chABSA-datasetの前処理

In [2]:
# ファイル一覧
data_dir = "bert_test/chABSA-dataset"
path_list = glob.glob(data_dir+'/*.json')
print("jsonファイル数 ",len(path_list))

jsonファイル数  230


In [3]:
# json確認
with open(path_list[0], "br") as f:
    j =  json.load(f)
sentences = j["sentences"]  # sentencesにはデータ数分のデータが格納されている。
print("1つ目のファイル内のデータ数チェック：", len(sentences))
print("sentences：", sentences)

1つ目のファイル内のデータ数チェック： 36
sentences： [{'sentence_id': 0, 'sentence': '当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・所得環境は改善し、景気も緩やかな回復基調のうちに推移いたしましたが、中国をはじめとするアジア新興国経済の減速懸念や、英国の欧州連合（ＥＵ）離脱決定、米国新政権への移行など、引き続き先行きは不透明な状況となっております', 'opinions': [{'target': 'わが国経済', 'category': 'NULL#general', 'polarity': 'neutral', 'from': 11, 'to': 16}, {'target': '企業業績', 'category': 'NULL#general', 'polarity': 'positive', 'from': 38, 'to': 42}, {'target': '雇用', 'category': 'NULL#general', 'polarity': 'positive', 'from': 43, 'to': 45}, {'target': '所得環境', 'category': 'NULL#general', 'polarity': 'positive', 'from': 46, 'to': 50}, {'target': '景気', 'category': 'NULL#general', 'polarity': 'positive', 'from': 55, 'to': 57}]}, {'sentence_id': 1, 'sentence': '当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節約志向により、個人消費は本格的な改善には至らず、また少子高齢化、人口減少による社会構造の変化、雇用改善に伴う労働コストの上昇、企業間競争の激化など、引き続き厳しい状況となりました', 'opinions': [{'target': '環境', 'category': 'NULL#general', 'polarity': 'negative', 'from': 11, 'to': 13}, {'target': '実質賃金', '

In [4]:
# idと文章のdf作成
def makeSentenceDf(path_list):
    sentenceDf = pd.DataFrame()
    for file in tqdm(path_list):
        with open(file, "br") as f:
            j =  json.load(f)
        sentences = j["sentences"]
        idsList = []
        sList = []
        for obj in sentences:
            ids = obj["sentence_id"]
            s = obj["sentence"]  #文章
            sList.append(s)
            idsList.append(ids)
        tmp = pd.DataFrame({'sentence_id':idsList, 'sentence':sList})
        sentenceDf = pd.concat([sentenceDf, tmp])
    return sentenceDf.reset_index(drop=True)

In [5]:
# df作成
sentenceDf = makeSentenceDf(path_list[:])
display(sentenceDf.head())

100%|███████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 180.81it/s]


,sentence_id,sentence
0,0,当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・...
1,1,当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節...
2,2,このような環境の中、当社グループはきのこ事業を中心として、健康食材である「きのこ」の研究開発...
3,3,また、平成26年に策定いたしました中期経営計画の見直しを行い、市況に左右されない事業ポートフ...
4,4,以上の結果、当連結会計年度の当社グループの業績は、売上高631億19百万円（前期比3.5％増...


In [6]:
# 丸囲い数字のコード取得
def get_marumoji(val):
    # ①を文字コードに変換[bytes型]
    maru_date = "①".encode("UTF8")
    # ①を文字コードに変換[int型]
    maru_code = int.from_bytes(maru_date, 'big')
    # 文字コードの変換
    maru_code += val - 1
    # 文字コードを文字に変換して返却
    return maru_code.to_bytes(4, "big").decode("UTF8")[1]

# 小文字化、記号・丸囲い数字除去、全角を半角など前処理関数
def preprocessing(text):    
    text = text.lower()  # 小文字化
    text = re.sub('\r\n','',text)  # \r\nをdelete
    text = re.sub(r'\d+','',text)  # 数字列をdelete
    ZEN = "".join(chr(0xff01 + i) for i in range(94))  # 全角文字列一覧
    HAN = "".join(chr(0x21 + i) for i in range(94))  # 半角文字列一覧
    ALL=re.sub(r'[a-zA-Zａ-ｚＡ-Ｚ\d]+','',ZEN+HAN)
    code_regex = re.compile('['+'〜'+'、'+'。'+'~'+'*'+'＊'+ALL+'「'+'」'+']')
    text = code_regex.sub(' ', text)  # 記号を消す
    
    maru_int = []
    for i in range(1,100):
        try:
            tmp = get_marumoji(i)
            maru_int.append(tmp)
        except UnicodeDecodeError:
            'UnicodeDecodeError, Skip a processing'
    code_regex = re.compile('['+"".join(i for i in maru_int)+']')
    text = code_regex.sub(' ', text)  # 丸囲い数字を消す
    
    ZEN = "".join(chr(0xff01 + i) for i in range(94))
    HAN = "".join(chr(0x21 + i) for i in range(94))
    ZEN2HAN = str.maketrans(ZEN, HAN)
    HAN2ZEN = str.maketrans(HAN, ZEN)
    text=text.translate(ZEN2HAN)  # 全角を半角に
    return text

In [7]:
# 前処理した文章をdfの新たな列に入れる
sentenceDf['preprocessingSentence'] = [preprocessing(text) for text in sentenceDf['sentence']]
sentenceDf = sentenceDf[sentenceDf['preprocessingSentence'].apply(lambda x:len(x)>0)]  # 何もない行は消す
display(sentenceDf.head())

,sentence_id,sentence,preprocessingSentence
0,0,当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・...,当連結会計年度におけるわが国経済は 政府の経済政策や日銀の金融緩和策により 企業業績 雇用・...
1,1,当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節...,当社グループを取り巻く環境は 実質賃金が伸び悩むなか 消費者の皆様の生活防衛意識の高まりや節...
2,2,このような環境の中、当社グループはきのこ事業を中心として、健康食材である「きのこ」の研究開発...,このような環境の中 当社グループはきのこ事業を中心として 健康食材である きのこ の研究開発...
3,3,また、平成26年に策定いたしました中期経営計画の見直しを行い、市況に左右されない事業ポートフ...,また 平成年に策定いたしました中期経営計画の見直しを行い 市況に左右されない事業ポートフォリ...
4,4,以上の結果、当連結会計年度の当社グループの業績は、売上高631億19百万円（前期比3.5％増...,以上の結果 当連結会計年度の当社グループの業績は 売上高億百万円 前期比 増 営業利益億...


In [8]:
# Mecabで分かち書きする関数
def wakachi(text, select_word_type=['名詞']):
    # MeCabの準備
    tagger = MeCab.Tagger()
    tagger.parse('')
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        word_type = node.feature.split(',')[0]
        if word_type in select_word_type:  # 指定した品詞のみ処理
            if word_type == '名詞':
                word_list.append(node.surface)
            else:
                word_list.append(node.feature.split(",")[6])
        else:
            pass
        node = node.next

    # リストを文字列に変換
    word_chain = ' '.join(word_list)
    if len(word_list)==0:
        'no list!'
    return word_list, word_chain  # リストでまとめた結果とすべてつなげた文字列を返す

# dfでまとめた文章をすべて分かち書き処理する関数
def makeWakachiDf(sentenceDf, col, stop_words, select_word_type=['名詞']):
    word_list_noun_list = []
    word_chain_noun_list = []
    for i, text in tqdm(enumerate(sentenceDf[col])):  # 指定した列にある文章を1行ずつ分かち書き処理
        word_list_noun, word_chain_noun = wakachi(text, select_word_type=select_word_type)
        word_list_noun = [w for w in word_list_noun if w not in stop_words]  # ストップワードも設定
        if len(word_list_noun)==0:  # 分かち書きとストップワード除去で何もなかった場合何もない結果を追加
            word_list_noun_list.append([])
            word_chain_noun_list.append('')
            continue
        word_list_noun_list.append(word_list_noun)  #分かち書きとストップワード除去の結果を追加
        word_chain_noun_list.append(word_chain_noun)  #分かち書きとストップワード除去の結果を追加
    return word_list_noun_list, word_chain_noun_list  # リストでまとめた結果とすべてつなげた文字列をリストにまとめて返す

In [9]:
# ストップワードを定義
with open('bert_test/Japanese.txt', 'r', encoding='utf-8') as f:
    stop_words = [w.strip() for w in f]
    stop_words = stop_words+['する', 'なる', 'いる', 'ある']
    stop_words = set(stop_words)
print(list(stop_words)[:10])

['あちら', '', '全部', '例', '力', '度', 'ふく', '区', 'さらい', 'みなさん']


In [10]:
# dfでまとめた文章をすべて分かち書き処理
word_list_noun_list, word_chain_noun_list = makeWakachiDf(sentenceDf, 'preprocessingSentence'
                                                          , stop_words, select_word_type=['名詞'])  # 今回は名詞だけ
# 分かち書き処理の結果をdfに追加
sentenceDf['wakachiSentenceList'] = word_list_noun_list
sentenceDf['wakachiSentenceChain'] = word_chain_noun_list
# 分かち書き処理の結果、空のリストの行を削除
sentenceDf = sentenceDf[sentenceDf['wakachiSentenceList'].apply(lambda x:len(x)>0)].reset_index(drop=True)
display(sentenceDf.head())

6119it [00:26, 227.03it/s]


,sentence_id,sentence,preprocessingSentence,wakachiSentenceList,wakachiSentenceChain
0,0,当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・...,当連結会計年度におけるわが国経済は 政府の経済政策や日銀の金融緩和策により 企業業績 雇用・...,"[連結会計, 年度, わが国, 経済, 政府, 経済政策, 日銀, 金融緩和, 策, 企業業...",連結会計 年度 わが国 経済 政府 経済政策 日銀 金融緩和 策 企業業績 雇用 所得 環境...
1,1,当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節...,当社グループを取り巻く環境は 実質賃金が伸び悩むなか 消費者の皆様の生活防衛意識の高まりや節...,"[当社, グループ, 環境, 実質賃金, 消費, 皆様, 生活防衛, 意識, 高まり, 節約...",当社 グループ 環境 実質賃金 なか 消費 者 皆様 生活防衛 意識 高まり 節約志向 個人...
2,2,このような環境の中、当社グループはきのこ事業を中心として、健康食材である「きのこ」の研究開発...,このような環境の中 当社グループはきのこ事業を中心として 健康食材である きのこ の研究開発...,"[環境, 当社, グループ, きのこ, 事業, 中心, 健康, 食材, きのこ, 研究開発,...",よう 環境 中 当社 グループ きのこ 事業 中心 健康 食材 きのこ 研究開発 生産 販売...
3,3,また、平成26年に策定いたしました中期経営計画の見直しを行い、市況に左右されない事業ポートフ...,また 平成年に策定いたしました中期経営計画の見直しを行い 市況に左右されない事業ポートフォリ...,"[平成, 策定, 中期経営計画, 見直し, 市況, 左右, 事業, ポートフォリオ, 構築,...",平成 年 策定 中期経営計画 見直し 市況 左右 事業 ポートフォリオ 構築 活動 推進 健...
4,4,以上の結果、当連結会計年度の当社グループの業績は、売上高631億19百万円（前期比3.5％増...,以上の結果 当連結会計年度の当社グループの業績は 売上高億百万円 前期比 増 営業利益億...,"[結果, 連結会計, 年度, 当社, グループ, 業績, 売上高, 前期比, 増, 営業利益...",以上 結果 連結会計 年度 当社 グループ 業績 売上高 億 百 万 円 前期比 増 営業利...


# Word2Vecの学習

In [11]:
%%time
# gensimでWord2Vecモデル作成
# vector_size : output dimention
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# epochs : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
wv_model = word2vec.Word2Vec(sentenceDf['wakachiSentenceList'].to_numpy()
                             , vector_size=200, min_count=5, window=5, sg=1)
# モデルの学習
wv_model.train(sentenceDf['wakachiSentenceList'].to_numpy(), total_examples=wv_model.corpus_count, epochs=20)
vocab = wv_model.wv.index_to_key  # 分かち書きした語彙
print(vocab[:5])

['事業', '年度', '連結会計', '百万円', '売上高']
Wall time: 3.22 s


# BERTの学習(MaskedLM)

In [11]:
# 東北大学の学習済みTokenizerを使用する
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# 東北大学の学習済みTokenizer読み込み
tokenizerOrg = BertJapaneseTokenizer.from_pretrained(model_name)
text_example = "当連結会計年度におけるわが国経済は 政府の経済政策や日銀の金融緩和策により 企業業績 雇用・所得環境は改善"
print(len(tokenizerOrg))  # トークンの数
print(tokenizerOrg.tokenize(text_example))  # 例文をトークンに分割
# chABSA-datasetをMecabで分けた結果の語彙をtokenizerに追加
addVocab = ' '.join(sentenceDf['wakachiSentenceChain'].to_list()).split(' ')
tokenizerOrg.add_tokens(addVocab)
print(len(tokenizerOrg))  # トークンの数
print(tokenizerOrg.tokenize(text_example))  # 例文をトークンに分割
tokenizerOrg.save_pretrained("bert_test/qiita_bert_token/")  # Save
# 注意：tokenizerOrg.vocab_sizeはOriginalのサイズを出力し、追加しても変わらないので注意（issue#4486）

32000
['当', '連結', '会計', '年度', 'における', 'わが', '##国', '経済', 'は', '政府', 'の', '経済', '政策', 'や', '日銀', 'の', '金融', '緩和', '策', 'により', '企業', '業績', '雇用', '・', '所得', '環境', 'は', '改善']
37138
['当', '連結会計', '年度', 'における', 'わが国', '経済', 'は', '政府', 'の', '経済政策', 'や', '日銀', 'の', '金融緩和', '策', 'により', '企業業績', '雇用', '・', '所得', '環境', 'は', '改善']


('bert_test/qiita_bert_token/tokenizer_config.json',
 'bert_test/qiita_bert_token/special_tokens_map.json',
 'bert_test/qiita_bert_token/vocab.txt',
 'bert_test/qiita_bert_token/added_tokens.json')

In [12]:
# モデルの構造を定義するBertConfigを設定
config = BertConfig(vocab_size=len(tokenizerOrg))  # 語彙数だけ設定してあとはデフォルト
print(config)
bertModel = BertForMaskedLM(config)  # MaskedLMタスクで学習

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 37138
}



In [13]:
# DataCollatorはいくつかの処理（パディングなど）を適用する
# DataCollatorForLanguageModelingなどは、形成されたバッチにランダムデータ拡張（ランダムマスキングなど）も適用
# ランダムに選ばれた15%のトークンを[MASK]という特殊トークンに置き換え、[MASK]にもともとあったトークンを予測するタスクで学習
# ただし、右のような確率でマスクする：80% MASK, 10% ランダムに他のトークンに, 10% 何も変更しない.
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizerOrg, mlm=True, mlm_probability=0.15)

In [14]:
# 学習するテキストをtxtファイルで保存しておく
display(sentenceDf['preprocessingSentence'].head(2))
sentenceDf['preprocessingSentence'].to_csv('bert_test/preprocessingSentence.txt', index=False, header=False)

0    当連結会計年度におけるわが国経済は 政府の経済政策や日銀の金融緩和策により 企業業績 雇用・...
1    当社グループを取り巻く環境は 実質賃金が伸び悩むなか 消費者の皆様の生活防衛意識の高まりや節...
Name: preprocessingSentence, dtype: object

In [15]:
# 1文書の最大長を定義
#with open('bert_test/preprocessingSentence.txt', encoding='utf-8') as f:
#    max_length = np.max([len(tokenizerOrg(line)['input_ids']) for line in f])
max_length = np.max([len(line) for line in tokenizerOrg(list(sentenceDf['preprocessingSentence'].to_numpy()))['input_ids']])
print('max length', max_length)

max length 166


In [16]:
# MaskedLMタスクで学習させるためのdatasetに加工
textDataset = LineByLineTextDataset(tokenizer=tokenizerOrg  # 使用するtokenizer
                                , file_path='bert_test/preprocessingSentence.txt'  # 学習するテキスト
                                , block_size=max_length, # tokenizerのmax_length
                               )

C:\Users\yusuke.kiridoshi\AppData\Local\pypoetry\Cache\virtualenvs\kiri-rM9h93NJ-py3.7\lib\site-packages\transformers\data\datasets\language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [17]:
# 事前学習に関するパラメータを設定
trainingArguments = TrainingArguments(output_dir= 'bert_test/'
                                      , overwrite_output_dir=True
                                      , num_train_epochs=3  # CPUなので少なめ
                                      #, per_device_train_batch_size=32
                                      #, save_steps=500
                                      #, save_strategy='steps'
                                      #, do_eval=True  # 評価データを用意する必要がある
                                      #, eval_steps=500  # 評価データを用意する必要がある
                                      #, evaluation_strategy='steps'  # 評価データを用意する必要がある
                                      #, load_best_model_at_end=True
                                      , logging_dir='bert_test/qiita_bert/runs/'
                                      , save_total_limit=3
                                      , prediction_loss_only=True
                                      , learning_rate=1e-3,  # CPUなので大きめ
                                     )
# 事前学習するインスタンス作成
trainer = Trainer(model=bertModel
                  , args=trainingArguments
                  , data_collator=data_collator
                  , train_dataset=textDataset
                 )

In [22]:
%%time
trainer.train()
trainer.save_model('bert_test/qiita_bert/')
# 3h 25min 33s

In [27]:
%load_ext tensorboard
%tensorboard --logdir 'bert_test/qiita_bert/runs/' --host=127.0.0.1

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# 評価データを入れる場合
'''
# 学習するテキストをtxtファイルで保存しておく
display(sentenceDf['preprocessingSentence'].head(2))
sentenceDf['preprocessingSentence'][:int(sentenceDf['preprocessingSentence'].shape[0]*0.8)].to_csv('bert_test/train.txt', index=False, header=False)
sentenceDf['preprocessingSentence'][int(sentenceDf['preprocessingSentence'].shape[0]*0.8):].to_csv('bert_test/valid.txt', index=False, header=False)
# MaskedLMタスクで学習させるためのdatasetに加工
textDatasetTrain = LineByLineTextDataset(tokenizer=tokenizerOrg  # 使用するtokenizer
                                , file_path='bert_test/train.txt'  # 学習するテキスト
                                , block_size=max_length, # tokenizerのmax_length
                               )
# MaskedLMタスクで学習させるためのdatasetに加工
textDatasetValid = LineByLineTextDataset(tokenizer=tokenizerOrg  # 使用するtokenizer
                                , file_path='bert_test/valid.txt'  # 学習するテキスト
                                , block_size=max_length, # tokenizerのmax_length
                               )

# 事前学習に関するパラメータを設定
trainingArguments = TrainingArguments(output_dir= 'bert_test/'
                                      , overwrite_output_dir=True
                                      , num_train_epochs=10  # CPUなので少なめ
                                      , per_device_train_batch_size=64
                                      , save_steps=200
                                      , save_strategy='steps'
                                      , do_eval=True  # 評価データを用意する必要がある
                                      , eval_steps=200  # 評価データを用意する必要がある
                                      , evaluation_strategy='steps'  # 評価データを用意する必要がある
                                      , load_best_model_at_end=True  # 評価データを用意する必要がある
                                      , logging_dir='bert_test/qiita_bert/runs/'
                                      , save_total_limit=3
                                      , prediction_loss_only=True
                                      , learning_rate=1e-3,  # CPUなので大きめ
                                     )
# 事前学習するインスタンス作成
trainer = Trainer(model=bertModel
                  , args=trainingArguments
                  , data_collator=data_collator
                  , train_dataset=textDatasetTrain
                  , eval_dataset=textDatasetValid
                 )
trainer.train()
trainer.save_model('bert_test/qiita_bert/')
'''

In [20]:
# 保存したBERTモデルを読み込み
bertModel = BertForMaskedLM.from_pretrained('bert_test/qiita_bert/')

loading configuration file bert_test/qiita_bert/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 37138
}

loading weights file bert_test/qiita_bert/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at bert_test/qiita_bert/.
If your task is similar to the task the model of the checkpoint was trained on, you can

In [21]:
# 前処理済みの文章をtokenizerでidに変換
encoded = tokenizerOrg.batch_encode_plus(sentenceDf['preprocessingSentence'].to_list()
                                         , padding=True
                                         , add_special_tokens=True)  # [CLS]のようなspecial tokenも含む
print(encoded.keys())
print(len(encoded['input_ids'][0]))  # input_idsの1文書の長さ
# BERTトークンID列を抽出
input_ids = torch.tensor(encoded['input_ids'])[:,:]
print(input_ids.shape)  # input_idsの型 文書数×1文書の長さ

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
166
torch.Size([6099, 166])


In [21]:
%%time
# 分散表現抽出
# 一気に処理するとMemory errorになるのでFor文で1行づつ抽出
last_hidden_states = torch.Tensor()
for inid in tqdm(input_ids[:]):
    bertModel.eval()
    with torch.no_grad():  # 勾配計算なし
        # 単語ベクトルを計算
        outputs = bertModel(inid.reshape(1,-1), output_hidden_states=True)  # 隠れ層の出力もさせる
    last_hidden_states = torch.cat((last_hidden_states, outputs['hidden_states'][-1]))  # 最後の隠れ層ベクトルを抽出
print(last_hidden_states.shape)
# 1h 36min 21s

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [1:35:12<00:00,  1.07it/s]

torch.Size([6099, 166, 768])
Wall time: 1h 35min 12s


In [ ]:
# 最後の隠れ層ベクトルsave
torch.save(last_hidden_states, 'bert_test/last_hidden_states01.pt')
# input_idsもsave
with open("bert_test/encoded01.pkl", "wb") as tf:
    pickle.dump(encoded, tf)
# tokenizersave
tokenizerOrg.save_pretrained('bert_test/tokenizer01')

# Word2Vec学習済みモデル

In [84]:
%%time
# Wikipediaで学習されたモデルをLoad
model_dir = 'bert_test/entity_vector/entity_vector.model.bin'
wv_modelPre = KeyedVectors.load_word2vec_format(model_dir, binary=True)
index2wordPre = wv_modelPre.index_to_key  # Wikipediaの語彙数

Wall time: 6.91 s


# BERTの学習済みモデル

In [22]:
# 東北大学の学習済みmodelを使用する
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# トークナイザーの読み込み
tokenizerPre = BertJapaneseTokenizer.from_pretrained(model_name)
# 学習済みモデルの読み込み
bertModelPre = BertModel.from_pretrained(model_name)

loading file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/vocab.txt from cache at C:\Users\yusuke.kiridoshi/.cache\huggingface\transformers\15164357d71cd32532e56c1d7c2757141326ae17c53e2277bc417cc7c21da6ea.a7378a0cbee5cff668832a776d72b97a25479604fe9564d5595897f75049e7f4
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/tokenizer_config.json from cache at C:\Users\yusuke.kiridoshi/.cache\huggingface\transformers\0e46f722799f19c3f0c53172545108a4b31847d3b9a2d5b100759f6673bd667b.08ae4e4044742b9cc7172698caf1da2524f5597ff8cf848114dd0b730cc44bdc
loading configuration file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-maskin

In [23]:
# 東北大学の学習済みTokenizerで分かち書き
encoded = tokenizerPre.batch_encode_plus(sentenceDf['preprocessingSentence'].to_list(), padding=True, add_special_tokens=True)
print(encoded.keys())
print(len(encoded['input_ids'][0]))  # input_idsの1文書の長さ
# 学習済みBERTトークンID列を抽出
input_ids = torch.tensor(encoded['input_ids'])[:,:]
print(input_ids.shape)  # input_idsの型 文書数×1文書の長さ

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
258
torch.Size([6099, 258])


In [24]:
%%time
# 分散表現抽出
# 一気に処理するとMemory errorになるのでFor文で1行づつ抽出
last_hidden_statesPre = torch.Tensor()
for inid in tqdm(input_ids[:]):
    with torch.no_grad():  # 勾配計算なし
        # 単語ベクトルを計算
        outputs = bertModelPre(inid.reshape(1,-1), output_hidden_states=True)
    last_hidden_statesPre = torch.cat((last_hidden_statesPre, outputs.last_hidden_state))
# 最終層の隠れ状態ベクトルを取得
print(last_hidden_statesPre.shape)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [2:10:46<00:00,  1.29s/it]

torch.Size([6099, 258, 768])
Wall time: 2h 10min 46s


In [ ]:
# 最後の隠れ層ベクトルsave
torch.save(last_hidden_statesPre, 'bert_test/last_hidden_states02.pt')

# WordNetの同義語取得

In [ ]:
'''
「8ケタの数字+"-(半角ハイフン)"+アルファベット」で表されています。アルファベットは品詞を表していて、
それぞれ
  n= 名詞
  v= 動詞
  a= 形容詞
  r= 副詞
  
同義語(synonym)
その概念を言い表すのに使用される語です。
英語のものは英語ワードネットのもので、日本語のものは複数の他言語ワードネットと対日辞書を用いて作成しました。
その後ある程度までは人手で修正しましたが、まだチェックしきれていないものも残っており、誤りが含まれている、
あるいは含まれるべきものが含まれていないという場合もございます。 同義語の扱いについての詳しい話はこちら。

関連synsetとのリンク
Hype　Hypernym　上位語　当該synsetが相手synsetに包含される　"canis familiaris"(02084071-n)は"domestic animal"(01317541-n)と"canid"(02083346-n)に包含される
Hypo　Hyponym　下位語　当該synsetが相手synsetを包含する　"canis familiaris"(02084071-n)は"toy canis familiaris"(02085374-n),"mutt"(02084861-n),"pooch"(02084732-n),...を包含する
Mprt　Meronyms --- Part　被構成要素(部分)　当該synsetが相手synsetという部分によって構成される　"canis familiaris"(02084071-n)は"flag"(02158846-n)を一部分として持つ
Hprt　Holonyms --- Part　構成要素(部分)　当該synsetが部分として相手synsetを構成する　"flag"(02158846-n)は"canis familiaris"(02084071-n)や"cervid"(02430045-n)の一部分である
Hmem　Holonyms --- Member　構成要素(構成員)　当該synsetが相手synsetの構成員である　"canis familiaris"(02084071-n)は"02083863-n"(canis)や"pack"(07994941-n)の構成員である
Mmem　Meronyms --- Member　被構成要素(構成員)　当該synsetが相手synsetという構成員によって構成される　"canis"(02083863-n)は"canis familiaris"(02084071-n)や" jackal"(02115096-n)、"wolf"(02114100-n)を構成員として持つ
Msub　Meronyms --- Substance　被構成要素(物質・材料)　当該synsetが相手synsetという物質or材料によって構成される　"ozone"(14972807-n)は"atomic number 8"(14648100-n)という物質を構成要素として持つ
Hsub　Holonyms --- Substance　構成要素(物質・材料)　当該synsetが相手synsetを構成する物質or材料である　"atomic number 8"(14648100-n)は"ozone"(14972807-n)や"water"(14845743-n)、"air"(14841267-n)を構成する物質である
Dmnc　Domain --- Category　被包含領域(カテゴリ)　当該synsetが相手synsetのカテゴリに属する　"comet"(09251407-n)は"astronomy"(06095022-n)のカテゴリに属する
Dmtc　In Domain --- Category　包含領域(カテゴリ)　当該synseが相手synsetが属するカテゴリである　"astronomy"(06095022-n)というカテゴリには"uprise"(01970348-v)や"absolute magnitude"(05090979-n)が属している
Dmnu　Domain --- Usage　被包含領域(語法)　当該synsetの用法が相手synsetの領域に限られる　"jean"(03594734-n)の用法は"plural form"(06295235-n)に限定される
Dmtu　In Domain --- Usage　包含領域(語法)　当該synsetの領域が相手synsetの用法を規定する　"plural form"(06295235-n)は"jean"(03594734-n)の用法を規定する
Dmnr　Domain --- Region　被包含領域(地域)　当該synsetが相手synsetの地域に属するものである　"sake"(07891433-n)は"nippon"(08921850-n)という地域に属している
Dmtr　In Domain --- Region　包含領域(地域)　当該synsetが相手synsetの属する地域である　"nippon"(08921850-n)は"sake"(07891433-n)の属する地域である
Inst　Instances　例　当該synsetは相手synsetの例である　"seiji ozawa"(11219502-n)は"director"(09952539-n)の例である
Hasi　Has Instance　例あり　当該synsetは相手synsetを例として持つ　"director"(09952539-n)は"seiji ozawa"(11219502-n)を例に持つ
Enta　Entails　含意　当該synsetを行う場合、必ず相手synsetも行っている　"fleece"(02319050-v)を行う場合、必ず"charge"(02320374-v)も行っている
Caus　Causes　引き起こし　当該synsetを行うと、相手synsetを引き起こす　"project"(02138075-v)を行うと、"appear"(00422090-v)を引き起こす
Also　See also　関連　当該synsetと相手synsetとの間に何らかの関連がある　"white"(00393105-a)は"light"(00408660-a)との間に何らかの関連がある
Attr　Attributes　属性　(a=形容詞のsynsetから見て)当該synsetが相手synsetという属性を表す際に使われる　"white"(00393105-a)は"value"(04979425-n)という属性を表す際に使われる
(n=名詞のsynsetから見て)当該synsetという属性を表す際に相手synsetを使う　"value"(04979425-n)という属性を表すのに"white"(00393105-a)を使う
Sim　Similar to　近似　当該synsetは表す意味が相手synsetと近似している　"white"(00393105-a)は意味が"albescent"(00393422-a)と近似している
'''

In [93]:
# WordNet DBと接続
conn = sqlite3.connect("bert_test/wnjpn.db")

In [95]:
# Table一覧取得
tableDf=pd.read_sql_query(
    """
    select
        name
    from
        sqlite_master
    where
        type='table'
    """
    , conn
)
display(tableDf)

,name
0,pos_def
1,link_def
2,synset_def
3,synset_ex
4,synset
5,synlink
6,ancestor
7,sense
8,word
9,variant


In [99]:
# wordidと単語一覧Table
# 日本語に限定
wordDf=pd.read_sql_query(
    """
    select
        *
    from
        word
    """
    , conn
)
wordDf = wordDf[wordDf['lang']=='jpn']
display(wordDf.head())

,wordid,lang,lemma,pron,pos
155287,155288,jpn,頭金,None,n
155288,155289,jpn,どうにかこうにか,None,r
155289,155290,jpn,大砲,None,n
155290,155291,jpn,スチーム,None,n
155291,155292,jpn,溢れでる,None,v


In [102]:
# 日本語の概念(上位語、被包含領域など)を表すsynsetとwordidを紐づけるためのTable
senseDf=pd.read_sql_query(
    """
    select
        *
    from
        sense
    """
    , conn
)
senseDf = senseDf[senseDf['lang']=='jpn']
display(senseDf.head())

,synset,wordid,lang,rank,lexid,freq,src
206941,13349834-n,155288,jpn,None,NaN,NaN,hand
206942,00151040-r,155289,jpn,None,NaN,NaN,hand
206943,02950632-n,155290,jpn,None,NaN,NaN,multi
206944,02746365-n,155290,jpn,None,NaN,NaN,hand
206945,02462828-n,155290,jpn,None,NaN,NaN,multi


In [104]:
# synsetの一覧Table
synsetDf=pd.read_sql_query(
    """
    select
        *
    from
        synset
    """
    , conn
)
display(synsetDf.head())

,synset,pos,name,src
0,07125096-n,n,expletive,eng30
1,07126228-n,n,expletive,eng30
2,14123044-n,n,measles,eng30
3,08030185-n,n,contras,eng30
4,09902017-n,n,beef_man,eng30


In [106]:
# 日本語の概念の関係の定義Table
synlinkDf=pd.read_sql_query(
    """
    select
        *
    from
        synlink
    """
    , conn
)
display(synlinkDf.head())

,synset1,synset2,link,src
0,07125096-n,07128527-n,hype,eng30
1,07126228-n,07109847-n,hype,eng30
2,14123044-n,14122235-n,hype,eng30
3,14123044-n,14123259-n,hypo,eng30
4,08030185-n,08197895-n,inst,eng30


In [108]:
# 概念の例文の定義Table
synset_defDf=pd.read_sql_query(
    """
    select
        *
    from
        synset_def
    """
    , conn
)
synset_defDf = synset_defDf[synset_defDf['lang']=='jpn']
display(synset_defDf.head())

,synset,lang,def,sid
118530,01785341-a,jpn,懇願によって静められない、なだめられない、または、動かされないさま,0
118532,11820323-n,jpn,南アフリカ産の茎のない多肉植物の属,0
118534,04239436-n,jpn,傷ついた前腕を支持する包帯,0
118536,04239436-n,jpn,首のあたりから掛かっている布の広い三角形の部分からなるもの,1
118538,11427067-n,jpn,太陽の大気圏の最も外側の領域,0


In [110]:
# 各テーブルを紐づける
# 各wordidにsynsetを追加
wordMaster = pd.merge(wordDf[['wordid', 'lemma', 'pos']], senseDf[['wordid', 'synset']], on=['wordid'], how='left')
display(wordMaster.head(1))
# 各synsetにnameを追加
wordMaster = pd.merge(wordMaster, synsetDf[['synset', 'name']], on=['synset'], how='left')
display(wordMaster.head(1))
# 各synsetに対応するwordidを追加
wordMaster = pd.merge(wordMaster, senseDf[['wordid', 'synset']].rename(columns={'wordid':'sameWordid'}), on=['synset'], how='left')
display(wordMaster.head(1))
# 各synsetに対応するwordidに対応するlemmaを追加
wordMaster = pd.merge(wordMaster
                      , wordDf[['wordid', 'lemma', 'pos']].rename(columns={'wordid':'sameWordid', 'lemma':'sameLemma', 'pos':'samePos'})
                      , on=['sameWordid'], how='left')
display(wordMaster.head(1))
# もともとのwordidと各synsetに対応するwordidが同じ行は削除
wordMaster = wordMaster[wordMaster['wordid']!=wordMaster['sameWordid']]
display(wordMaster.head())

,wordid,lemma,pos,synset
0,155288,頭金,n,13349834-n


,wordid,lemma,pos,synset,name
0,155288,頭金,n,13349834-n,deposit


,wordid,lemma,pos,synset,name,sameWordid
0,155288,頭金,n,13349834-n,deposit,155288


,wordid,lemma,pos,synset,name,sameWordid,sameLemma,samePos
0,155288,頭金,n,13349834-n,deposit,155288,頭金,n


,wordid,lemma,pos,synset,name,sameWordid,sameLemma,samePos
1,155288,頭金,n,13349834-n,deposit,161058,デポジット,n
2,155288,頭金,n,13349834-n,deposit,165962,手付け金,n
3,155288,頭金,n,13349834-n,deposit,183741,内金,n
4,155288,頭金,n,13349834-n,deposit,194952,証拠金,n
5,155288,頭金,n,13349834-n,deposit,247271,手金,n


In [ ]:
# こっちでも可
wordMaster =pd.read_sql_query(
    """
    select
        wordid
        , lemma
        , pos
        , lang
        , synset
        , name
        , t_merge02.sameWordid as sameWordid
        , sameLemma
        , samePos
    from
        (
        select
            t_merge01.wordid as wordid
            , lemma
            , pos
            , lang
            , t_merge01.synset as synset
            , name
            , sameWordid
        from
            (
            select
                t_word.wordid as wordid
                , lemma
                , pos
                , lang
                , synset
            from
                (
                select
                    wordid
                    , lemma
                    , pos
                    , lang
                from
                    word
                where
                    lang='jpn'
                ) as t_word
            left join
                (
                select
                    wordid
                    , synset
                from
                    sense
                where
                    lang='jpn'
                ) as t_sense
            on
                t_word.wordid = t_sense.wordid
            ) as t_merge01
        left join
            (
            select
                synset
                , name
            from
                synset
            ) as t_synset
        on
            t_merge01.synset = t_synset.synset
        left join
            (
            select
                wordid as sameWordid
                , synset
            from
                sense
            where
                lang='jpn'
            ) as t_sense02
        on
            t_merge01.synset = t_sense02.synset
        ) as t_merge02
    left join
        (
        select
            wordid as sameWordid
            , lemma as sameLemma
            , pos as samePos
        from
            word
        where
            lang='jpn'
        ) as t_word02
    on
        t_merge02.sameWordid = t_word02.sameWordid
    where
        wordid<>t_merge02.sameWordid
    """
    , conn
)
display(wordMaster.head())

# 同義語の抽出の評価

## chABSA-dataset Word2Vec評価

In [119]:
# Word2Vec学習時に定義した語彙のリスト vocabに出てくる単語を評価対象にする
# 同義語抽出評価

sameWordsCnt = 0  # 初期値 同義語の数
# vocabに出てくる単語に限定
wordMasterPart = wordMaster[(wordMaster['lemma'].isin(list(vocab)))&(wordMaster['sameLemma'].isin(list(vocab)))]
# vocabに出てくる単語に限定した後のWordNetの同義語の数
allSameWordsCnt = wordMasterPart[['lemma', 'sameLemma']].drop_duplicates().shape[0]
print('限定したWordNetの語彙数', wordMasterPart['lemma'].unique().shape[0])
print('限定したWordNetの単語にあるすべての同義語の数', allSameWordsCnt, '\n')
# 限定したWordNetの単語ごとの同義語についてモデルがいくつ同じ同義語を抽出できているかカウントする
for i, tango in enumerate(wordMasterPart['lemma'].unique()):
    # tango=WordNetの単語1つずつループ
    # tangoに最も似ている単語Top5をdfにまとめる
    w2vCosDf = pd.DataFrame(wv_model.wv.most_similar(tango), columns=['sameLemma', 'cos'])
    w2vCosDf.insert(0, 'lemma', tango)
    # 限定したWordNetと紐づける
    mergeDf = pd.merge(wordMasterPart[wordMasterPart['lemma']==tango][['lemma', 'sameLemma']].drop_duplicates()
                       , w2vCosDf, on=['lemma', 'sameLemma'], how='inner')
    # 紐づけたdfの結果
    if len(mergeDf) > 0:
        print(tango)
        display(mergeDf)
        print('===============', '\n')
    sameWordsCnt += len(mergeDf)  # 紐づけられた同義語の数を足す
# 最終的な正解数
print('正解数', str(sameWordsCnt), '/', allSameWordsCnt)

限定したWordNetの語彙数 787
限定したWordNetの単語にあるすべての同義語の数 2810 

創造


,lemma,sameLemma,cos
0,創造,創出,0.804618



取扱


,lemma,sameLemma,cos
0,取扱,取扱い,0.696676



船舶


...


つゆ


,lemma,sameLemma,cos
0,つゆ,スープ,0.898817



モデル


,lemma,sameLemma,cos
0,モデル,フレーム,0.602747



正解数 79 / 2810


## chABSA-dataset BERT評価

In [ ]:
# gensimと違って自分で似ている単語を抽出する必要がある
# 分散表現のcos類似度を計算して、最も近い単語Top5を抽出する

In [138]:
# 最後の隠れ層ベクトルLoad
last_hidden_states = torch.load('bert_test/last_hidden_states01.pt')
# input_idsもLoad
with open("bert_test/encoded01.pkl", "rb") as tf:
    encoded = pickle.load(tf)
input_ids = torch.tensor(encoded.input_ids)

tokenizerOrg = BertJapaneseTokenizer.from_pretrained('bert_test/tokenizer01')

loading file bert_test/tokenizer01\vocab.txt
loading file bert_test/tokenizer01\added_tokens.json
loading file bert_test/tokenizer01\special_tokens_map.json
loading file bert_test/tokenizer01\tokenizer_config.json
Adding 連結会計 to the vocabulary
Adding わが国 to the vocabulary
Adding 経済政策 to the vocabulary
Adding 金融緩和 to the vocabulary
Adding 企業業績 to the vocabulary
Adding 新興国 to the vocabulary
Adding 欧州連合 to the vocabulary
Adding eu to the vocabulary
Adding 新政権 to the vocabulary
Adding 先行き to the vocabulary
Adding 実質賃金 to the vocabulary
Adding 皆様 to the vocabulary
Adding 生活防衛 to the vocabulary
Adding 節約志向 to the vocabulary
Adding 個人消費 to the vocabulary
Adding 少子高齢化 to the vocabulary
Adding 人口減少 to the vocabulary
Adding 社会構造 to the vocabulary
Adding 企業間 to the vocabulary
Adding きのこ to the vocabulary
Adding 研究開発 to the vocabulary
Adding 事業活動 to the vocabulary
Adding 中期経営計画 to the vocabulary
Adding 市況 to the vocabulary
Adding ポートフォリオ to the vocabulary
Adding 邁進 to the vocabulary
Adding 売上高 to 

Adding amata to the vocabulary
Adding ポリネシアン・サンライトカーニバル to the vocabulary
Adding aloha to the vocabulary
Adding rapanui to the vocabulary
Adding ポリネシアン・グランドステージ to the vocabulary
Adding ‵ to the vocabulary
Adding aheo to the vocabulary
Adding あなたの笑顔 to the vocabulary
Adding アクアリウム to the vocabulary
Adding フィッシュゴーランド to the vocabulary
Adding 話題性 to the vocabulary
Adding 一巡 to the vocabulary
Adding 太平洋・島サミット to the vocabulary
Adding お客様 to the vocabulary
Adding リニューアルオープン to the vocabulary
Adding pacific to the vocabulary
Adding フラガール to the vocabulary
Adding aaa to the vocabulary
Adding 西島隆弘 to the vocabulary
Adding 貸切バス to the vocabulary
Adding 団体旅行 to the vocabulary
Adding 福島県沖地震 to the vocabulary
Adding 電力会社 to the vocabulary
Adding 運輸業 to the vocabulary
Adding 輸送量 to the vocabulary
Adding 減益 to the vocabulary
Adding 経済状況 to the vocabulary
Adding 有料老人ホーム to the vocabulary
Adding 賃料 to the vocabulary
Adding 建設工事 to the vocabulary
Adding 舶用 to the vocabulary
Adding 過年度 to the vocabular

Adding キシルテック to the vocabulary
Adding エクセレントスイーツシリーズ to the vocabulary
Adding 小箱 to the vocabulary
Adding ビスケット to the vocabulary
Adding キャンデー to the vocabulary
Adding 伸張 to the vocabulary
Adding 米菓 to the vocabulary
Adding 発酵バター to the vocabulary
Adding 抹茶 to the vocabulary
Adding パイン to the vocabulary
Adding いも to the vocabulary
Adding マイベネフィット to the vocabulary
Adding カカオ to the vocabulary
Adding オリーブオイル to the vocabulary
Adding ココナッツオイル to the vocabulary
Adding ショコラ to the vocabulary
Adding きなこ to the vocabulary
Adding アルフォートミニチョコレート to the vocabulary
Adding ブリリアント to the vocabulary
Adding トリュフ to the vocabulary
Adding 神戸学院大学 to the vocabulary
Adding 神戸市 to the vocabulary
Adding 産学官連携 to the vocabulary
Adding バニラ to the vocabulary
Adding チーズおかき to the vocabulary
Adding フェットチーネ to the vocabulary
Adding グミ to the vocabulary
Adding ミックスフルーツソーダ to the vocabulary
Adding はちみつレモン to the vocabulary
Adding 新味 to the vocabulary
Adding フェットチーネグミコーラ to the vocabulary
Adding ハードボイルド to the voc

Adding ltda to the vocabulary
Adding ニッシン to the vocabulary
Adding アジノモト to the vocabulary
Adding メントス to the vocabulary
Adding 中国大陸 to the vocabulary
Adding 味道 to the vocabulary
Adding 華南 to the vocabulary
Adding 出前一丁 to the vocabulary
Adding 香港国際空港 to the vocabulary
Adding マイカップヌードルファクトリー to the vocabulary
Adding ファクトリー to the vocabulary
Adding アジア地域 to the vocabulary
Adding 個人所得 to the vocabulary
Adding 資源価格 to the vocabulary
Adding 値動き to the vocabulary
Adding 下振れリスク to the vocabulary
Adding 年明け to the vocabulary
Adding 合理化 to the vocabulary
Adding 即席麺 to the vocabulary
Adding 全社 to the vocabulary
Adding 粉粒体 to the vocabulary
Adding 顆粒 to the vocabulary
Adding 生ラーメン to the vocabulary
Adding 経済財政政策 to the vocabulary
Adding 内需 to the vocabulary
Adding 構造改革 to the vocabulary
Adding 焼肉のたれ to the vocabulary
Adding 生鮮 to the vocabulary
Adding 新製品開発 to the vocabulary
Adding 手羽 to the vocabulary
Adding 赤身肉 to the vocabulary
Adding 熟成肉 to the vocabulary
Adding 甘口 to the vocabulary
Adding 黒酢

Adding 絶対数 to the vocabulary
Adding ℓ to the vocabulary
Adding エスキューブ to the vocabulary
Adding エスタンク to the vocabulary
Adding ありません to the vocabulary
Adding 金融緩和政策 to the vocabulary
Adding 印刷物 to the vocabulary
Adding 紙媒体 to the vocabulary
Adding 競争激化 to the vocabulary
Adding dnp to the vocabulary
Adding モビリティ to the vocabulary
Adding printing to the vocabulary
Adding information to the vocabulary
Adding 利便性 to the vocabulary
Adding 本人確認 to the vocabulary
Adding サイバートラスト株式会社 to the vocabulary
Adding 協業 to the vocabulary
Adding ポータルサイト to the vocabulary
Adding 地域通貨 to the vocabulary
Adding 株式会社フューチャーリンクネットワーク to the vocabulary
Adding sig to the vocabulary
Adding エスアイジー to the vocabulary
Adding コンビブロックグループ to the vocabulary
Adding 日本市場 to the vocabulary
Adding ブラインド to the vocabulary
Adding シェード to the vocabulary
Adding 耐候性 to the vocabulary
Adding 耐摩耗性 to the vocabulary
Adding 地球環境 to the vocabulary
Adding 効果的 to the vocabulary
Adding 採光 to the vocabulary
Adding 消費電力 to the vocabulary
A

...

Adding 事業計画 to the vocabulary
Adding 石炭火力発電 to the vocabulary
Adding 電力供給 to the vocabulary
Adding lng to the vocabulary
Adding 急落 to the vocabulary
Adding マイカ to the vocabulary
Adding クローラー to the vocabulary
Adding サインシステム to the vocabulary
Adding トピレックプラザ to the vocabulary
Adding 東京都江東区南砂 to the vocabulary
Adding スポーツクラブ to the vocabulary
Adding osso to the vocabulary
Adding 日立ツール株式会社 to the vocabulary
Adding 三菱日立ツール株式会社 to the vocabulary
Adding 発行済 to the vocabulary
Adding 三菱マテリアル株式会社 to the vocabulary
Adding 一般管理費 to the vocabulary
Adding 工具鋼 to the vocabulary
Adding 経営資源 to the vocabulary
Adding 日立金属 to the vocabulary
Adding グン to the vocabulary
Adding 南通 to the vocabulary
Adding 射出成形 to the vocabulary
Adding 軟質 to the vocabulary
Adding 磁性材料 to the vocabulary
Adding アモルファス to the vocabulary
Adding 希土類磁石 to the vocabulary
Adding 電動パワーステアリング to the vocabulary
Adding ハイブリッド自動車 to the vocabulary
Adding フェライト to the vocabulary
Adding 鋳物 to the vocabulary
Adding 農業機械 to the vocabulary
A

Adding 引当 to the vocabulary
Adding ms to the vocabulary
Adding エイジア to the vocabulary
Adding 債権回収 to the vocabulary
Adding 補助金 to the vocabulary
Adding 慰労 to the vocabulary
Adding 借主 to the vocabulary
Adding 連帯保証人 to the vocabulary
Adding 小林達也 to the vocabulary
Adding 東京地方裁判所 to the vocabulary
Adding 破産手続き to the vocabulary
Adding 損害賠償 to the vocabulary
Adding 会社法 to the vocabulary
Adding 賠償金 to the vocabulary
Adding 西日本宮入販売株式会社 to the vocabulary
Adding 供給契約 to the vocabulary
Adding 債務不履行 to the vocabulary
Adding 不法行為 to the vocabulary
Adding 一部請求 to the vocabulary
Adding 売買代金 to the vocabulary
Adding 反訴 to the vocabulary
Adding 東京高等裁判所 to the vocabulary
Adding 係属 to the vocabulary
Adding リレーションシップ to the vocabulary
Adding lpg to the vocabulary
Adding 貯槽 to the vocabulary
Adding 新事業 to the vocabulary
Adding 食品加工 to the vocabulary
Adding サニタリー to the vocabulary
Adding 液体窒素 to the vocabulary
Adding カバーニャ・グループ to the vocabulary
Adding カバーニャ to the vocabulary
Adding 持株会社 to the vocabulary


Adding 二酸化マンガン to the vocabulary
Adding 導電性高分子 to the vocabulary
Adding 有価証券 to the vocabulary
Adding 制裁金 to the vocabulary
Adding 独占禁止法 to the vocabulary
Adding fanuc to the vocabulary
Adding field to the vocabulary
Adding intelligent to the vocabulary
Adding edge to the vocabulary
Adding link to the vocabulary
Adding 栃木県壬生町 to the vocabulary
Adding cnc to the vocabulary
Adding サーボモータ to the vocabulary
Adding 茨城県筑西市 to the vocabulary
Adding スピードアップ to the vocabulary
Adding 協働 to the vocabulary
Adding ファナック to the vocabulary
Adding cr to the vocabulary
Adding 財団法人 to the vocabulary
Adding 大河内 to the vocabulary
Adding レーザ to the vocabulary
Adding ロボマシン to the vocabulary
Adding ロボドリル to the vocabulary
Adding 切削加工 to the vocabulary
Adding 一時的 to the vocabulary
Adding ロボショット to the vocabulary
Adding ロボカット to the vocabulary
Adding 放電加工 to the vocabulary
Adding 大統領選 to the vocabulary
Adding 防衛省 to the vocabulary
Adding 東海旅客鉄道株式会社 to the vocabulary
Adding jr東海 to the vocabulary
Adding リニア中央新幹

Adding シアトルマリナーズ to the vocabulary
Adding 運営会社 to the vocabulary
Adding civic to the vocabulary
Adding フルモデルチェンジ to the vocabulary
Adding 体質改善 to the vocabulary
Adding 裁断 to the vocabulary
Adding 新機種 to the vocabulary
Adding 切替 to the vocabulary
Adding リコール to the vocabulary
Adding usドル to the vocabulary
Adding ⇒ to the vocabulary
Adding nsx to the vocabulary
Adding ts to the vocabulary
Adding tech to the vocabulary
Adding alabama to the vocabulary
Adding トリム to the vocabulary
Adding avancier to the vocabulary
Adding gienia to the vocabulary
Adding 広州市 to the vocabulary
Adding 増城区 to the vocabulary
Adding 開発計画 to the vocabulary
Adding 生産ライン to the vocabulary
Adding 製造物流 to the vocabulary
Adding 導線 to the vocabulary
Adding bangladesh to the vocabulary
Adding 模倣品 to the vocabulary
Adding チェーン店 to the vocabulary
Adding 倹約 to the vocabulary
Adding 不動産市場 to the vocabulary
Adding 過熱感 to the vocabulary
Adding 資本流出 to the vocabulary
Adding brexit to the vocabulary
Adding 顕在化 to the vocabulary


Adding 武田薬品工業株式会社 to the vocabulary
Adding 大阪市中央 to the vocabulary
Adding 共同出資 to the vocabulary
Adding 創薬 to the vocabulary
Adding ベンチャー企業 to the vocabulary
Adding スコヒアファーマ to the vocabulary
Adding 神奈川県藤沢市 to the vocabulary
Adding ピークアウト to the vocabulary
Adding サプライチェーン to the vocabulary
Adding アトル to the vocabulary
Adding 福岡市東区 to the vocabulary
Adding 株式会社メディセオ to the vocabulary
Adding 埼玉県三郷市 to the vocabulary
Adding エバルス to the vocabulary
Adding 岡山県 to the vocabulary
Adding 都窪郡 to the vocabulary
Adding flc to the vocabulary
Adding 品切れ to the vocabulary
Adding presus to the vocabulary
Adding プレサス to the vocabulary
Adding mr to the vocabulary
Adding 認定試験 to the vocabulary
Adding 専門知識 to the vocabulary
Adding pms to the vocabulary
Adding area to the vocabulary
Adding front to the vocabulary
Adding pharmacy to the vocabulary
Adding real to the vocabulary
Adding time to the vocabulary
Adding support to the vocabulary
Adding 需要予測 to the vocabulary
Adding 在庫管理 to the vocabulary
Adding オー

In [135]:
# input_idsと分散表現を連結
summaryOrg = torch.cat((input_ids.reshape(input_ids.shape[0], input_ids.shape[1], 1)
                        , last_hidden_states), dim=2)
print(summaryOrg.shape)

torch.Size([6099, 166, 769])


In [145]:
%%time
# 共通する語彙の分散表現だけ集計
commonVocabs = wordMasterPart['lemma'].unique()  # 語彙
nonDelVocab = tokenizerOrg.encode(list(commonVocabs))  # WordNetにある単語のトークン
delVocab = [0.,1.,2.,3.,4.]  # 特殊トークン
summaryOrgTs = torch.Tensor()
for ts in tqdm(summaryOrg[:,:,:]):
    tmp = pd.DataFrame(ts[:,:].numpy())  #1行目の各トークンの分散表現
    tmp = tmp[tmp[0].isin(nonDelVocab)]  # WordNetにある単語のトークンは消さない
    tmp = tmp[~tmp[0].isin(delVocab)]  # 特殊トークン消す
    summaryOrgTs = torch.cat((summaryOrgTs, torch.tensor(tmp.to_numpy())))
display(tmp.head())  # 1列目はトークンid、2列目以降は分散表現
print(summaryOrgTs.shape)

100%|██████████████████████████████████████████████████████████████████████████████| 6099/6099 [01:53<00:00, 53.84it/s]


,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
1,20525.0,-0.861274,0.18735,0.307756,0.020265,1.788669,-0.316896,-1.036553,0.56695,-0.691072,...,-2.849060,0.025727,1.643805,0.131071,1.233061,-2.362948,-1.608163,-0.891338,-0.069695,-0.461069
2,5161.0,-0.861274,0.18735,0.307756,0.020265,1.788669,-0.316896,-1.036553,0.56695,-0.691072,...,-2.849060,0.025727,1.643805,0.131071,1.233061,-2.362948,-1.608163,-0.891339,-0.069695,-0.461069
9,14461.0,-0.861274,0.18735,0.307756,0.020265,1.788669,-0.316896,-1.036554,0.56695,-0.691072,...,-2.849059,0.025727,1.643805,0.131071,1.233061,-2.362948,-1.608163,-0.891339,-0.069695,-0.461069
15,701.0,-0.861274,0.18735,0.307756,0.020265,1.788669,-0.316896,-1.036553,0.56695,-0.691072,...,-2.849060,0.025727,1.643805,0.131071,1.233061,-2.362948,-1.608163,-0.891338,-0.069695,-0.461069
16,1346.0,-0.861274,0.18735,0.307756,0.020265,1.788669,-0.316896,-1.036553,0.56695,-0.691072,...,-2.849060,0.025727,1.643805,0.131071,1.233061,-2.362948,-1.608163,-0.891338,-0.069695,-0.461069


torch.Size([38317, 769])
Wall time: 1min 53s


In [146]:
# 複数の分散表現がある単語は平均とる
summaryOrgDf = pd.DataFrame(summaryOrgTs.numpy())
summaryOrgTsMean = torch.tensor(summaryOrgDf.groupby([0]).mean().reset_index().to_numpy())
# 語彙数
# Word2vecの方は787個
print(summaryOrgTsMean.shape[0])

781


In [149]:
# Word2Vec学習時に定義した語彙のリスト vocabに出てくる単語を評価対象にする
# 各トークンのcos類似度が最も高いTop5のトークンを取得
# 同義語抽出評価

sameWordsCnt = 0  # 初期値 同義語の数
num=0  # カウントの変数
print('限定したWordNetの語彙数', wordMasterPart['lemma'].unique().shape[0])
print('限定したWordNetの語彙と共通する語彙数', summaryOrgTsMean[:,1:].shape[0])  # 限定したWordNetの語彙と共通する語彙数
print('限定したWordNetの単語にあるすべての同義語の数', allSameWordsCnt, '\n')
# 限定したWordNetの単語ごとの同義語についてモデルがいくつ同じ同義語を抽出できているかカウントする
for i, v1 in tqdm(enumerate(summaryOrgTsMean[:,1:])):
    # i行目のトークンidをdecode
    tango = tokenizerOrg.decode(int(summaryOrgTsMean[i,0].item())).replace(' ','')  # 変な空白が入っているときがあるので削除
    tagger = MeCab.Tagger()
    noun = re.split('[\t,]', tagger.parse(tango))[1]  # 品詞取得
    if noun!='名詞':  # 名詞以外のときはスキップ
        continue
    # 単語v1とv2のcos類似度を計算する
    cosList = []
    for j, v2 in (enumerate(summaryOrgTsMean[:,1:])):
        if i==j:
            cosList.append(0)
        else:
            cosList.append(1 - scipy.spatial.distance.cosine(v1, v2))  # cos類似度
    # cos類似度top5 Token取得
    top5Token = summaryOrgTsMean[list(np.argsort(cosList)[-5:][::-1]),0].tolist()
    top5Cos = [cosList[co] for co in list(np.argsort(cosList)[-5:][::-1])]
    top5 = [(tokenizerOrg.decode(int(to)).replace(' ',''), co) for to, co in zip(top5Token, top5Cos)]
    # tangoに最も似ている単語Top5をdfにまとめる
    tmp = pd.DataFrame(top5
                       , columns=['sameLemma', 'cos'])
    tmp.insert(0, 'lemma', tango)
    # 限定したWordNetと紐づける
    mergeDf = pd.merge(wordMasterPart[wordMasterPart['lemma']==tango][['lemma', 'sameLemma']].drop_duplicates()
                       , tmp, on=['lemma', 'sameLemma'], how='inner')
    # 紐づけたdfの結果
    if len(mergeDf) > 0:
        print(tango)
        display(mergeDf)
        print('===============', '\n')
    sameWordsCnt += len(mergeDf)  # 紐づけられた同義語の数を足す
    num += 1
# 最終的な正解数
print('正解数', str(sameWordsCnt), '/', allSameWordsCnt)

限定したWordNetの語彙数 787
限定したWordNetの語彙と共通する語彙数 781
限定したWordNetの単語にあるすべての同義語の数 2810 



15it [00:00, 27.44it/s]

種


,lemma,sameLemma,cos
0,種,材,1.0


21it [00:00, 26.58it/s]

...


貸出


,lemma,sameLemma,cos
0,貸出,リース,1.0


664it [00:25, 24.83it/s]

781it [00:30, 25.62it/s]

正解数 28 / 2810


## Wikipedia学習済みWord2Vec評価

In [161]:
# Word2Vec学習時に定義した語彙のリスト vocabに出てくる単語を評価対象にする
# 同義語抽出評価

sameWordsCnt = 0  # 初期値 同義語の数
print('限定したWordNetの語彙数', wordMasterPart['lemma'].unique().shape[0])
# 重複する単語を抽出して長さ確認
print('限定したWordNetの語彙と共通する語彙数'
      , len([k for k, v in collections.Counter(list(wordMasterPart['lemma'].unique())+index2wordPre).items() if v > 1]))
print('限定したWordNetの単語にあるすべての同義語の数', allSameWordsCnt, '\n')
# 限定したWordNetの単語ごとの同義語についてモデルがいくつ同じ同義語を抽出できているかカウントする
for i, tango in tqdm(enumerate(wordMasterPart['lemma'].unique())):
    try:
        # tango=WordNetの単語1つずつループ
        # tangoに最も似ている単語Top5をdfにまとめる
        w2vCosDf = pd.DataFrame(wv_modelPre.most_similar(tango), columns=['sameLemma', 'cos'])
        w2vCosDf.insert(0, 'lemma', tango)
        # 限定したWordNetと紐づける
        mergeDf = pd.merge(wordMasterPart[wordMasterPart['lemma']==tango][['lemma', 'sameLemma']].drop_duplicates()
                           , w2vCosDf, on=['lemma', 'sameLemma'], how='inner')
        # 紐づけたdfの結果(数が多いので出力しない)
        #if len(mergeDf) > 0:
        #    print(tango)
        #    display(mergeDf)
        #    print('===============', '\n')
        sameWordsCnt += len(mergeDf)  # 紐づけられた同義語の数を足す
    except KeyError:
        'A tango not present'
# 最終的な正解数
print('正解数', str(sameWordsCnt), '/', allSameWordsCnt)

限定したWordNetの語彙数 787
限定したWordNetの語彙と共通する語彙数 782
限定したWordNetの単語にあるすべての同義語の数 2810 



787it [00:44, 17.54it/s]

正解数 426 / 2810


## Wikipedia学習済みBERT評価

In [162]:
# いろいろ読み込み

# 最後の隠れ層ベクトルLoad
last_hidden_states = torch.load('bert_test/last_hidden_states02.pt')

# 東北大学の学習済みmodelを使用する
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# トークナイザーの読み込み
tokenizerPre = BertJapaneseTokenizer.from_pretrained(model_name)
# 学習済みモデルの読み込み
bertModelPre = BertModel.from_pretrained(model_name)
# 東北大学の学習済みTokenizerで分かち書き
encoded = tokenizerPre.batch_encode_plus(sentenceDf['preprocessingSentence'].to_list(), padding=True, add_special_tokens=True)
print(encoded.keys())
print(len(encoded['input_ids'][0]))  # input_idsの1文書の長さ
# 学習済みBERTトークンID列を抽出
input_ids = torch.tensor(encoded['input_ids'])[:,:]
print(input_ids.shape)  # input_idsの型 文書数×1文書の長さ

loading file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/vocab.txt from cache at C:\Users\yusuke.kiridoshi/.cache\huggingface\transformers\15164357d71cd32532e56c1d7c2757141326ae17c53e2277bc417cc7c21da6ea.a7378a0cbee5cff668832a776d72b97a25479604fe9564d5595897f75049e7f4
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/tokenizer_config.json from cache at C:\Users\yusuke.kiridoshi/.cache\huggingface\transformers\0e46f722799f19c3f0c53172545108a4b31847d3b9a2d5b100759f6673bd667b.08ae4e4044742b9cc7172698caf1da2524f5597ff8cf848114dd0b730cc44bdc
loading configuration file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-maskin

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
258
torch.Size([6099, 258])


In [163]:
# input_idsと分散表現を連結
summaryOrgPre = torch.cat((input_ids.reshape(input_ids.shape[0], input_ids.shape[1], 1)
                           , last_hidden_states), dim=2)
print(summaryOrgPre.shape)

torch.Size([6099, 258, 769])


In [164]:
%%time
# 共通する語彙の分散表現だけ集計
nonDelVocab = tokenizerPre.encode(list(commonVocabs))  # WordNetにある単語のトークン
delVocab = [0.,1.,2.,3.,4.]  # 特殊トークン
summaryOrgTsPre = torch.Tensor()
for ts in tqdm(summaryOrgPre[:,:,:]):
    tmp = pd.DataFrame(ts[:,:].numpy())  #1行目の各トークンの分散表現
    tmp = tmp[tmp[0].isin(nonDelVocab)]  # WordNetにある単語のトークンは消さない
    tmp = tmp[~tmp[0].isin(delVocab)]  # 特殊トークン消す
    summaryOrgTsPre = torch.cat((summaryOrgTsPre, torch.tensor(tmp.to_numpy())))
display(tmp.head())  # 1列目はトークンid、2列目以降は分散表現
print(summaryOrgTsPre.shape)

Token indices sequence length is longer than the specified maximum sequence length for this model (789 > 512). Running this sequence through the model will result in indexing errors
100%|██████████████████████████████████████████████████████████████████████████████| 6099/6099 [02:34<00:00, 39.58it/s]


,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
1,20525.0,-0.468271,0.749700,-0.195274,-0.249739,-0.499166,-0.072858,0.105265,0.004123,-0.028559,...,-0.198605,-0.194617,0.102433,0.397646,-0.095702,-0.642062,-0.211784,-0.393812,-1.054616,0.392896
2,5161.0,-0.394134,0.603517,0.310727,-0.329336,-0.618172,0.061398,0.102205,0.118264,0.401545,...,-0.324053,-0.292790,0.214343,0.387949,0.199548,-0.768709,-0.009901,-0.413048,-1.160358,0.262009
3,1790.0,-0.321748,0.631709,0.006191,-0.340471,-0.251261,-0.385065,0.024060,0.172158,0.331330,...,-0.151674,-0.108270,0.100474,0.453578,0.067145,-0.789796,-0.125366,-0.339459,-1.022954,0.194144
4,5161.0,-0.242368,0.544518,0.270213,-0.370365,-0.563863,0.015539,0.023054,-0.032608,0.436251,...,-0.206094,-0.402527,0.337104,0.260721,0.113517,-0.846188,0.169125,-0.277054,-1.064100,0.378227
10,14461.0,-0.517592,0.699880,0.002472,-0.252257,-0.665148,-0.155570,0.149252,0.078627,0.208231,...,-0.136309,-0.147250,-0.083597,0.295493,0.209624,-0.770756,-0.044984,-0.271981,-1.164710,0.711935


torch.Size([52080, 769])
Wall time: 2min 34s


In [165]:
# 複数の分散表現がある単語は平均とる
summaryOrgDfPre = pd.DataFrame(summaryOrgTsPre.numpy())
summaryOrgTsPreMean = torch.tensor(summaryOrgDfPre.groupby([0]).mean().reset_index().to_numpy())
# 語彙数
# Word2vecの方は787個
print(summaryOrgTsPreMean.shape[0])

718


In [166]:
# Word2Vec学習時に定義した語彙のリスト vocabに出てくる単語を評価対象にする
# 各トークンのcos類似度が最も高いTop5のトークンを取得
# 同義語抽出評価

sameWordsCnt = 0  # 初期値 同義語の数
num=0  # カウントの変数
print('限定したWordNetの語彙数', wordMasterPart['lemma'].unique().shape[0])
print('限定したWordNetの語彙と共通する語彙数', summaryOrgTsPreMean[:,1:].shape[0])  # 限定したWordNetの語彙と共通する語彙数
print('限定したWordNetの単語にあるすべての同義語の数', allSameWordsCnt, '\n')
# 限定したWordNetの単語ごとの同義語についてモデルがいくつ同じ同義語を抽出できているかカウントする
for i, v1 in tqdm(enumerate(summaryOrgTsPreMean[:,1:])):
    # i行目のトークンidをdecode
    tango = tokenizerOrg.decode(int(summaryOrgTsPreMean[i,0].item())).replace(' ','')  # 変な空白は言っているときがあるので削除
    tagger = MeCab.Tagger()
    noun = re.split('[\t,]', tagger.parse(tango))[1]  # 品詞取得
    if noun!='名詞':  # 名詞のときはスキップ
        continue
    # 単語v1とv2のcos類似度を計算する
    cosList = []
    for j, v2 in (enumerate(summaryOrgTsPreMean[:,1:])):
        if i==j:
            cosList.append(0)
        else:
            cosList.append(1 - scipy.spatial.distance.cosine(v1, v2))  # cos類似度
    # cos類似度top5 Token取得
    top5Token = summaryOrgTsPreMean[list(np.argsort(cosList)[-5:][::-1]),0].tolist()
    top5Cos = [cosList[co] for co in list(np.argsort(cosList)[-5:][::-1])]
    top5 = [(tokenizerOrg.decode(int(to)).replace(' ',''), co) for to, co in zip(top5Token, top5Cos)]
    # tangoに最も似ている単語Top5をdfにまとめる
    tmp = pd.DataFrame(top5
                       , columns=['sameLemma', 'cos'])
    tmp.insert(0, 'lemma', tango)
    # 限定したWordNetと紐づける
    mergeDf = pd.merge(wordMasterPart[wordMasterPart['lemma']==tango][['lemma', 'sameLemma']].drop_duplicates()
                       , tmp, on=['lemma', 'sameLemma'], how='inner')
    # 紐づけたdfの結果(数が多いので出力しない)
    #if len(mergeDf) > 0:
    #    print(tango)
    #    display(mergeDf)
    #    print('===============', '\n')
    sameWordsCnt += len(mergeDf)  # 紐づけられた同義語の数を足す
    num += 1
# 最終的な正解数
print('正解数', str(sameWordsCnt), '/', allSameWordsCnt)

限定したWordNetの語彙数 787
限定したWordNetの語彙と共通する語彙数 718
限定したWordNetの単語にあるすべての同義語の数 2810 



718it [00:26, 26.72it/s]


正解数 673 / 2810
